# 🤖 Demonstração Avançada de Agentes com Gymnasium (Padrão OpenAI)

Este notebook atende à solicitação de usar uma biblioteca de agentes mais sofisticada e fácil de usar, padrão na indústria de IA. Usaremos a **Gymnasium**, a sucessora da famosa biblioteca `Gym` da OpenAI.

**O que faremos de diferente:**
1.  **Criaremos nosso próprio Ambiente:** Modelaremos o "Mundo do Aspirador de Pó" como um ambiente customizado na Gymnasium. Isso é fundamental para aplicar IA a problemas novos.
2.  **Implementaremos os Agentes:** Mostraremos como cada tipo de agente (Simples de Reflexo, Baseado em Modelo e de Aprendizado) interage com este ambiente padronizado.

Isso conecta a teoria dos slides diretamente com a prática moderna de Aprendizado por Reforço (RL).

## ⚙️ 1. Preparação: Instalação das Bibliotecas

Precisamos da `gymnasium` para criar o ambiente e da `numpy` para nos ajudar com a tabela do agente de aprendizado.

In [ ]:
!pip install gymnasium numpy

## 🧱 2. Construindo o Ambiente do Aspirador de Pó

Para que um agente possa interagir, primeiro precisamos definir as 'regras do jogo'. Um ambiente na Gymnasium precisa ter:

- **Espaço de Ações (`action_space`):** Quais ações o agente pode tomar?
  - `0`: Ir para a Esquerda
  - `1`: Ir para a Direita
  - `2`: Aspirar
- **Espaço de Observações (`observation_space`):** O que o agente pode 'ver' do ambiente?
  - Vamos definir os estados possíveis: `(Localização, Status de A, Status de B)`
- **Função `reset()`:** Reinicia o ambiente para um estado inicial.
- **Função `step(action)`:** Executa uma ação, atualiza o ambiente e retorna a nova observação, a recompensa, e se o episódio terminou.

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random

class VacuumCleanerEnv(gym.Env):
    """Ambiente customizado do Aspirador de Pó seguindo a interface da Gymnasium."""
    
    def __init__(self):
        super(VacuumCleanerEnv, self).__init__()
        
        # Definir o espaço de ações: 0: Esquerda, 1: Direita, 2: Aspirar
        self.action_space = spaces.Discrete(3)
        
        # Definir o espaço de observação: [localização (0=A, 1=B), status_A (0=Limpo, 1=Sujo), status_B]
        self.observation_space = spaces.MultiDiscrete([2, 2, 2])
        
        self.state = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        # Estado inicial aleatório para melhor treinamento
        self.state = self.observation_space.sample()
        return np.array(self.state), {}

    def step(self, action):
        location, status_A, status_B = self.state
        reward = -1  # Custo de -1 para cada ação (incentiva a eficiência)
        
        if action == 0: # Ir para a Esquerda
            self.state[0] = 0 # Vai para A
        elif action == 1: # Ir para a Direita
            self.state[0] = 1 # Vai para B
        elif action == 2: # Aspirar
            if location == 0 and status_A == 1: # Se está em A e A está sujo
                self.state[1] = 0 # Limpa A
                reward = 10 # Recompensa grande por limpar
            elif location == 1 and status_B == 1: # Se está em B e B está sujo
                self.state[2] = 0 # Limpa B
                reward = 10 # Recompensa grande por limpar
            else:
                reward = -5 # Penalidade por tentar aspirar um local limpo
        
        # O episódio termina quando ambos os locais estão limpos
        terminated = (self.state[1] == 0 and self.state[2] == 0)
        
        return np.array(self.state), reward, terminated, False, {}

    def render(self, mode='human'):
        print("Estado atual:",self.state)
        location, status_A, status_B = self.state
        sala_a = "(A, Sujo)" if status_A == 1 else "(A, Limpo)"
        sala_b = "(B, Sujo)" if status_B == 1 else "(B, Limpo)"
        
        if location == 0:
            print(f"[*] {sala_a}  |  {sala_b}")
        else:
            print(f"  {sala_a}  |  [*] {sala_b}")

# Testando se o ambiente funciona
env = VacuumCleanerEnv()


---

## 🧠 3. Agente Simples de Reflexo

Este agente é uma política "hard-coded". Ele não aprende e não tem memória. Ele simplesmente mapeia uma observação (`state`) para uma ação, seguindo regras fixas.

In [ ]:
def simple_reflex_policy(observation):
    """Define a política de um agente simples de reflexo."""
    location, status_A, status_B = observation
    
    # Se o local atual está sujo, aspire
    if location == 0 and status_A == 1:
        return 2 # Aspirar
    if location == 1 and status_B == 1:
        return 2 # Aspirar
    
    # Se o local atual está limpo, mova-se
    if location == 0:
        return 1 # Ir para a Direita
    else:
        return 0 # Ir para a Esquerda

# Executando o agente simples
print("Executando Agente Simples de Reflexo:")
observation, info = env.reset()
terminated = False
total_reward = 0

while not terminated:
    env.render()
    action = simple_reflex_policy(observation)
    observation, reward, terminated, _, _ = env.step(action)
    total_reward += reward
    print(f"Ação Tomada: {['Esquerda', 'Direita', 'Aspirar'][action]} | Recompensa: {reward}")

print("\nTarefa concluída!")
env.render()
print(f"Recompensa Total: {total_reward}")

---

## 📝 4. Agente de Reflexo Baseado em Modelo

Este agente é um pouco mais inteligente. Ele constrói um **modelo interno** do mundo para evitar ações desnecessárias. Por exemplo, se ele já sabe que a sala A está limpa, ele não precisa ir até lá para verificar novamente.

In [ ]:
class ModelBasedAgent:
    def __init__(self):
        # O modelo armazena o que o agente sabe sobre o mundo
        # None = Desconhecido, 0 = Limpo, 1 = Sujo
        self.model = {'A': None, 'B': None}
    
    def choose_action(self, observation):
        location, status_A, status_B = observation
        
        # Atualiza o modelo com a percepção atual
        self.model['A'] = status_A
        self.model['B'] = status_B
        
        print(f"Modelo interno do agente: {self.model}")
        
        # Se o local atual está sujo, aspire
        if location == 0 and status_A == 1:
            return 2 # Aspirar
        if location == 1 and status_B == 1:
            return 2 # Aspirar
            
        # Se o local atual está limpo, verifique o modelo para decidir para onde ir
        if location == 0: # Estou em A, que está limpo
            if self.model['B'] != 0: # Se não sei que B está limpo, vou para lá
                return 1 # Direita
        else: # Estou em B, que está limpo
            if self.model['A'] != 0: # Se não sei que A está limpo, vou para lá
                return 0 # Esquerda
        
        # Se ambos os locais são conhecidos como limpos, a tarefa terminou (o loop principal vai parar)
        return random.choice([0,1]) # Ação padrão se não há mais o que fazer

# Executando o agente baseado em modelo
print("Executando Agente Baseado em Modelo:")
agent = ModelBasedAgent()
observation, info = env.reset()
terminated = False

while not terminated:
    env.render()
    action = agent.choose_action(observation)
    observation, reward, terminated, _, _ = env.step(action)
    print(f"Ação Tomada: {['Esquerda', 'Direita', 'Aspirar'][action]}")
    
print("\nTarefa concluída!")
env.render()

---

## 🎓 5. Agente Baseado em Aprendizado (Q-Learning)

Este é o agente mais avançado. Ele não tem nenhuma regra pré-definida. Ele **aprende** a melhor política de ações por tentativa e erro, usando as recompensas do ambiente. Usaremos o algoritmo Q-Learning, conforme a fórmula do slide 49. 

$$Q(s, a) \leftarrow Q(s, a) + \alpha [r + \gamma \max_{a'} Q(s', a') - Q(s, a)]$$

O estado `s` aqui será um número único para cada uma das 8 combinações de observação `(loc, stat_A, stat_B)`.

In [ ]:
# Hiperparâmetros para o treinamento
num_episodes = 10
learning_rate = 0.1 # Alpha
discount_factor = 0.99 # Gamma
epsilon = 1.0 # Probabilidade de exploração (começa alta)
min_epsilon = 0.05
epsilon_decay_rate = 0.9995

# A tabela Q terá uma linha para cada estado e uma coluna para cada ação.
# Como nosso espaço de observação é [2, 2, 2], temos 2*2*2 = 8 estados possíveis.
q_table = np.zeros((env.observation_space.nvec.prod(), env.action_space.n))

def observation_to_state(obs):
    """Converte um array de observação [l, sA, sB] em um único inteiro."""
    return obs[0] * 4 + obs[1] * 2 + obs[2]

print("Iniciando o treinamento do Agente Q-Learning...")

# Loop de treinamento
for episode in range(num_episodes):
    observation, info = env.reset()
    state = observation_to_state(observation)
    terminated = False

    while not terminated:
        # Exploração vs. Explotação (Epsilon-greedy)
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Ação aleatória (explorar)
        else:
            action = np.argmax(q_table[state]) # Melhor ação conhecida (explorar)
        
        new_observation, reward, terminated, _, _ = env.step(action)
        new_state = observation_to_state(new_observation)
        
        # Atualização da Tabela Q
        old_value = q_table[state, action]
        next_max = np.max(q_table[new_state])
        
        new_value = (1 - learning_rate) * old_value + learning_rate * (reward + discount_factor * next_max)
        q_table[state, action] = new_value
        
        state = new_state
    
    # Reduzir epsilon para diminuir a exploração ao longo do tempo
    if epsilon > min_epsilon:
        epsilon *= epsilon_decay_rate

print("Treinamento Concluído!")
print(f"Epsilon final: {epsilon:.3f}")

### Avaliando o Agente Treinado

Agora, vamos desativar a exploração (`epsilon = 0`) e ver o agente usar a política que aprendeu para resolver o problema de forma ótima.

In [ ]:
print("\nExecutando o Agente Q-Learning treinado (modo de avaliação):")
observation, info = env.reset()
terminated = False
total_reward = 0

while not terminated:
    env.render()
    state = observation_to_state(observation)
    action = np.argmax(q_table[state]) # Sempre escolhe a melhor ação
    observation, reward, terminated, _, _ = env.step(action)
    total_reward += reward
    print(f"Ação Tomada: {['Esquerda', 'Direita', 'Aspirar'][action]} | Recompensa: {reward}")

print("\nTarefa concluída de forma ótima!")
env.render()
print(f"Recompensa Total: {total_reward}")

## ✅ Conclusão

Neste notebook, usamos a biblioteca `Gymnasium`, padrão da indústria, para explorar os conceitos da aula:

1.  **Agente Simples de Reflexo:** Uma política fixa e reativa, fácil de implementar, mas ineficiente.
2.  **Agente Baseado em Modelo:** Melhora a eficiência ao manter um estado interno, mas ainda depende de lógica pré-programada.
3.  **Agente de Aprendizado (Q-Learning):** O mais poderoso. Não precisa de regras; ele **aprende a política ótima** interagindo com o ambiente, tornando-se capaz de resolver problemas complexos de forma autônoma.

Ferramentas como a Gymnasium são a base para treinar agentes em tarefas muito mais complexas, desde jogos como Xadrez e Go até controle de robôs e veículos autônomos. 